# Computational Theory Tasks

This notebook contains the solutions to the tasks for the Computational Theory module.

## Task 1: Binary Representations

This task implements various binary manipulation functions as specified in the requirements.

In [4]:
import unittest

def rotl(x, n=1):
    """Rotate bits in a 32-bit unsigned integer to the left by n places."""
    n %= 32  # Ensure n is within valid range
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF

def rotr(x, n=1):
    """Rotate bits in a 32-bit unsigned integer to the right by n places."""
    n %= 32  # Ensure n is within valid range
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

def ch(x, y, z):
    """Choose bits from y where x has bits set to 1, otherwise take bits from z."""
    return (x & y) | ((~x & 0xFFFFFFFF) & z)

def maj(x, y, z):
    """Majority function: output has a 1 where at least two of x, y, and z have 1's in that position."""
    return (x & y) | (x & z) | (y & z)

### Testing the binary functions

In [9]:
# Test cases for rotl function
print("Testing rotl function:")
test_value = 0b10110011
result = rotl(test_value, 2)
print(f"rotl(0b{test_value:08b}, 2) = 0b{result:08b}")
print(f"Expected: 0b11001101")
print(f"Correct: {result == 0b11001101}")
print()

# Test cases for rotr function
print("Testing rotr function:")
test_value = 0b10110011
result = rotr(test_value, 2)
print(f"rotr(0b{test_value:08b}, 2) = 0b{result:08b}")
print(f"Expected: 0b11101100")
print(f"Correct: {result == 0b11101100}")
print()

# Test cases for ch function
print("Testing ch function:")
x, y, z = 0b10110011, 0b11001100, 0b11110000
result = ch(x, y, z)
print(f"ch(0b{x:08b}, 0b{y:08b}, 0b{z:08b}) = 0b{result:08b}")
print(f"Expected: 0b11001100")
print(f"Correct: {result == 0b11001100}")
print()

# Test cases for maj function
print("Testing maj function:")
x, y, z = 0b10110011, 0b11001100, 0b11110000
result = maj(x, y, z)
print(f"maj(0b{x:08b}, 0b{y:08b}, 0b{z:08b}) = 0b{result:08b}")
print(f"Expected: 0b11110000")
print(f"Correct: {result == 0b11110000}")

Testing rotl function:
rotl(0b10110011, 2) = 0b1011001100
Expected: 0b11001101
Correct: False

Testing rotr function:
rotr(0b10110011, 2) = 0b11000000000000000000000000101100
Expected: 0b11101100
Correct: False

Testing ch function:
ch(0b10110011, 0b11001100, 0b11110000) = 0b11000000
Expected: 0b11001100
Correct: False

Testing maj function:
maj(0b10110011, 0b11001100, 0b11110000) = 0b11110000
Expected: 0b11110000
Correct: True


## Task 2: Hash Functions

This task involves implementing the hash function from *The C Programming Language* by Brian Kernighan and Dennis Ritchie in Python.

In [ ]:
def hash(s: str) -> int:
    """Convert the C hash function to Python.
    
    Original C implementation:
    unsigned hash(char *s) {
        unsigned hashval;
        for (hashval = 0; *s != '\0'; s++)
            hashval = *s + 31 * hashval;
        return hashval % 101;
    }
    """
    hashval = 0
    for c in s:
        hashval = ord(c) + 31 * hashval
    return hashval % 101

### Testing the hash function

In [ ]:
# Test the hash function with some sample strings
test_strings = ["hello", "world", "python", "hash", "function", "computational", "theory"]

print("Testing hash function:")
for s in test_strings:
    print(f"hash(\"{s}\") = {hash(s)}")

### Explanation of values 31 and 101

- **31 is chosen as the multiplier** because it is a prime number. Using a prime number as a multiplier helps to distribute the hash values more evenly across the hash table. The number 31 specifically has been empirically shown to work well as a hash multiplier, producing fewer collisions than many other values. It's also a Mersenne prime (2^5 - 1), which means it can be computed efficiently in binary (31 * n = 32 * n - n = n << 5 - n).

- **101 is used for the modulo operation** because it's also a prime number. When using the modulo operation to fit hash values into a hash table, using a prime number helps minimize collisions. If the modulus were a composite number with factors shared by common character sequences, it would increase the chance of collisions. The number 101 is chosen as it's a reasonable size for a small hash table that balances memory usage with collision avoidance.

## Task 3: SHA256

This task involves implementing a function to calculate the SHA256 padding for a given file.

In [11]:
def calculate_sha256_padding(filepath: str) -> bytes:
    """Calculate the SHA256 padding for a given file.
    
    The SHA256 padding consists of:
    1. A single '1' bit
    2. Enough '0' bits so the length in bits of padded message 
       is the smallest possible multiple of 512
    3. The length in bits of the original input as a big-endian 64-bit unsigned integer
    """
    with open(filepath, 'rb') as f:
        content = f.read()
    
    # Calculate message length in bits
    size_bits = len(content) * 8
    
    # Calculate number of padding bits needed
    # We need to add 1 bit + padding_bits + 64 bits = 512 * k
    padding_bits = (448 - (size_bits + 1)) % 512
    if padding_bits < 0:
        padding_bits += 512
        
    # Create the padding
    # First byte contains a 1 bit followed by 7 zero bits (0x80 in hex)
    padding = bytearray([0x80])
    # Add the padding zero bytes
    padding.extend([0] * (padding_bits // 8))
    # Add the original message length as a 64-bit big-endian integer
    padding.extend(size_bits.to_bytes(8, byteorder='big'))
    
    return bytes(padding)

def print_padding_hex(padding: bytes):
    """Print the padding in hexadecimal format."""
    hex_str = ' '.join(f'{b:02x}' for b in padding)
    # Format output with 26 bytes per line to match the example
    bytes_per_line = 26
    for i in range(0, len(hex_str), bytes_per_line * 3):
        print(hex_str[i:i + bytes_per_line * 3])

In [12]:
# Create a test file with the content "abc"
with open('text.txt', 'w') as f:
    f.write("abc")

# Calculate and print the padding
padding = calculate_sha256_padding('text.txt')
print("SHA256 padding for 'abc':")
print_padding_hex(padding)

SHA256 padding for 'abc':
80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
00 00 00 00 00 00 00 00 18


## Task 4: Prime Numbers

This task involves calculating the first 100 prime numbers using two different algorithms.

In [16]:
def trial_division():
    """Calculate the first 100 prime numbers using the trial division method.
    
    Trial division works by checking if a number is divisible by any integer from 2 up to n-1.
    If no divisors are found, the number is prime.
    """
    # Define a nested function to check if a number is prime
    def is_prime(n):
        if n < 2:
            return False  # Numbers less than 2 are not prime
        # An optimisation: we only need to check divisors up to sqrt(n)
        for i in range(2, int(n**0.5) + 1):
            if n % i == 0:
                return False  # If n is divisible by any number between 2 and sqrt(n), it is not prime
        return True  # If no divisors are found, n is prime

    primes = []  # Initialise an empty list to store prime numbers
    num = 2  # Start checking for primes from the number 2
    while len(primes) < 100:  # Continue until we have found 100 prime numbers
        if is_prime(num):
            primes.append(num)  # If num is prime, add it to the list
        num += 1 
    return primes  # Return the list of prime numbers

def sieve():
    """Calculate the first 100 prime numbers using the Sieve of Eratosthenes.
    
    This algorithm works by iteratively marking the multiples of each prime number as composite,
    starting from 2. The remaining unmarked numbers are prime.
    """
    # 550 is chosen because the 100th prime is 541
    sieve = [True] * 550  # Create a list of boolean values, all set to True
    sieve[0] = sieve[1] = False  # Set the first two values (0 and 1) to False, as they are not prime
    
    primes = []  # Initialise an empty list to store prime numbers
    for i in range(2, 550):
        if sieve[i]:
            primes.append(i)  # If sieve[i] is True, i is a prime number
            # Mark all multiples of i as False (not prime)
            # We can start from i*i because all smaller multiples have already been marked
            for j in range(i * i, 550, i):
                sieve[j] = False
        if len(primes) == 100:
            break  # Stop once we have found 100 prime numbers
    return primes  # Return the list of prime numbers

In [17]:
# Print results from both methods
print("Trial Division method:")
trial_division_primes = trial_division()
print(trial_division_primes)
print("\nSieve method:")
sieve_primes = sieve()
print(sieve_primes)

# Verify that both methods produce the same result
print("\nDo both methods produce the same result?", trial_division_primes == sieve_primes)

Trial Division method:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]

Sieve method:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]

Do both methods p

### Explanation of the Algorithms

#### Trial Division
Trial division is the simplest primality test. It works by checking if a number n is divisible by any integer from 2 up to √n (we only need to check up to the square root because if n = a × b, at least one of a or b is ≤ √n).

**Advantages**:
- Simple to understand and implement
- Works for any range of numbers

**Disadvantages**:
- Inefficient for large numbers or long ranges
- Has a time complexity of O(n√n) when finding all primes up to n

#### Sieve of Eratosthenes
The Sieve of Eratosthenes is an ancient algorithm for finding all prime numbers up to a specified limit. It works by iteratively marking the multiples of each prime, starting from 2. The numbers that remain unmarked are prime.

**Advantages**:
- Much more efficient than trial division for finding all primes in a range
- Has a time complexity of O(n log log n) when finding all primes up to n

**Disadvantages**:
- Requires more memory to store the sieve array
- Less efficient if you only need to check a few specific numbers

## Task 5: Roots

This task involves calculating the first 32 bits of the fractional part of the square roots of the first 100 prime numbers.

In [ ]:
def get_first_100_primes():
    """Calculate the first 100 prime numbers using the Sieve of Eratosthenes."""
    # We'll use the sieve method as it's more efficient
    sieve = [True] * 550  # 550 is chosen because the 100th prime is 541
    sieve[0] = sieve[1] = False
    
    primes = []
    for i in range(2, 550):
        if sieve[i]:
            primes.append(i)
            for j in range(i * i, 550, i):
                sieve[j] = False
        if len(primes) == 100:
            break
    return primes

def get_binary_fraction(decimal_fraction, bits=32):
    """Convert a decimal fraction to its binary representation with the specified number of bits.
    
    The algorithm works by repeatedly multiplying the decimal fraction by 2 and taking the integer part
    as the next bit in the binary representation. This is analogous to the way we convert a decimal
    number to binary, but for the fractional part.
    """
    binary = ""
    for _ in range(bits):
        decimal_fraction *= 2
        bit = int(decimal_fraction)
        binary += str(bit)
        decimal_fraction -= bit
    return binary

def calculate_root_bits():
    """Calculate the first 32 bits of the fractional part of the square roots of the first 100 prime numbers."""
    primes = get_first_100_primes()
    results = []
    
    for prime in primes:
        # Calculate square root and get fractional part
        sqrt_value = pow(prime, 0.5)
        fractional_part = sqrt_value - int(sqrt_value)
        
        # Get binary representation of the fractional part
        binary = get_binary_fraction(fractional_part)
        results.append((prime, binary))
    
    return results

In [ ]:
# Calculate and print the results
results = calculate_root_bits()

# Print the first 5 and last 5 results for brevity
print("First 5 results:")
for prime, binary in results[:5]:
    print(f"√{prime} = {int(pow(prime, 0.5))}.{binary}")

print("\nLast 5 results:")
for prime, binary in results[-5:]:
    print(f"√{prime} = {int(pow(prime, 0.5))}.{binary}")